In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.mam.*
import jline.solvers.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)

// Create closed job class with 10 jobs, starting at delay station
val jobclass = ClosedClass(model, "Class1", 10, delay, 0)

In [ ]:
// Set service processes
delay.setService(jobclass, Exp.fitMean(1.0))  // mean = 1
queue1.setService(jobclass, Exp.fitMean(1.5)) // mean = 1.5

In [ ]:
// Set routing matrix
val P = model.initRoutingMatrix()
P.set(jobclass, jobclass, delay, delay, 0.7)
P.set(jobclass, jobclass, delay, queue1, 0.3)
P.set(jobclass, jobclass, queue1, delay, 1.0)
P.set(jobclass, jobclass, queue1, queue1, 0.0)

model.link(P)

In [ ]:
// Create array of different solvers to compare
val solvers = arrayOf(
    CTMC(model, "keep", true),
    JMT(model, "seed", 23000, "verbose", true, "keep", true),
    SSA(model, "seed", 23000, "verbose", true, "samples", 5000),
    FLD(model),
    MVA(model),
    NC(model, "method", "exact"),
    MAM(model),
    LINE(model)
)

// Execute each solver and display results
for (solver in solvers) {
    try {
        val avgTable = solver.getAvgTable()
        avgTable.print()
    } catch (e: Exception) {
        println("Solver failed: ${e.message}")
    }
}

This repairmen model demonstrates:

1. **Closed Network**: 10 jobs circulate between two stations
2. **Delay Station**: Models customer thinking time (mean = 1.0)
3. **Service Station**: Models actual service (mean = 1.5)
4. **Probabilistic Routing**: 70% chance of staying at delay, 30% to service
5. **Multiple Solvers**: Different solution approaches for validation

The model represents a typical machine repair scenario where:
- Machines break down and need repair
- There's a delay between breakdowns (thinking time)
- Repairs follow exponential service times
- Multiple solution methods provide consistent results